In [1]:
# Data Analysis Libraries
import pandas as pd
import numpy as np
import math
import string
import time

# Web Scrapping Libraries
import requests
from bs4 import BeautifulSoup

#pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns',None)

In [2]:
def dados_iniciais_livro(i):
    
    url_resenha = 'https://www.skoob.com.br/livro/resenhas/{}/mais-gostaram/mpage:1'.format(i) 
    # página do Skoob em que constam as resenhas, em que o primeiro {} será o número do livro e o segundo {} a página de reviews
    
    response = requests.get(url_resenha.format(i))
    pagina_html = BeautifulSoup(response.text)
    corpo = pagina_html.findAll(id="corpo")[0] # corpo da página a ser feita a raspagem de dados (Web Scraping)    
    
    Livro = corpo.findAll('strong')[0].get_text() # Nome do Livro
    
    try:
        Autor = corpo.findAll('a')[1].get_text() # Nome do Autor     
    except:
        Autor = None
    if Autor == '\nR$ \n':
        Autor = corpo.findAll('i')[0].get_text()
        
    try:
        ISBN_13 = corpo.findAll('div', {"class": "sidebar-desc"})[0].find('span').get_text()
    except:
        ISBN_13 = None
        
    try:
        lancamento = int(corpo.findAll('div', {"class": "sidebar-desc"})[0].get_text().split('Ano: ')[-1].split(' / Páginas:',1)[0])
    except:
        lancamento = None
    
    Paginas = corpo.findAll('div', {"class": "sidebar-desc"})[0].get_text().split('Páginas: ')[-1].split(' ',1)[0]    

    try:
        total_resenhas = int(corpo.findAll(id="perfil-conteudo-intern")[0].findAll('b')[0].get_text().split(' ')[0])
    except:
        total_resenhas = 0
    
    print('Livro {}: {}'.format(i, Livro))
        
    return(Livro, Autor, ISBN_13, lancamento, Paginas, total_resenhas, url_resenha)

In [3]:
def detalhes_do_livro(i, Livro):
    
    titulo_tratado = Livro
    for c in string.punctuation:
        titulo_tratado = titulo_tratado.replace(c,'')
    
    url_livro = ('https://www.skoob.com.br/'+titulo_tratado+'-'+str(i)).replace(' ','-').lower().replace('--','-')
    
    response2 = requests.get(url_livro)
    pagina_html2 = BeautifulSoup(response2.text)
    corpo2 = pagina_html2.findAll(id="corpo")[0] # corpo da página a ser feita a raspagem de dados (Web Scraping)
    detalhes = corpo2.findAll(id='pg-livro-principal-container')[0]
    
    Rating = float(detalhes.find(id="pg-livro-box-rating").find('span').get_text())
    Avaliacoes = int(detalhes.find(id="pg-livro-box-rating-avaliadores-numero").get_text().split(' ')[0].replace('.',''))
    
    total_resenhas_real = int(detalhes.findAll('div',{'class':'bar'})[0].findAll('a')[1].get_text().replace('.',''))
    abandonos = int(detalhes.findAll('div',{'class':'bar'})[1].findAll('a')[1].get_text().replace('.',''))
    relendo = int(detalhes.findAll('div',{'class':'bar'})[2].findAll('a')[1].get_text().replace('.',''))
    querem_ler = int(detalhes.findAll('div',{'class':'bar'})[3].findAll('a')[1].get_text().replace('.',''))
    lendo = int(detalhes.findAll('div',{'class':'bar'})[4].findAll('a')[1].get_text().replace('.',''))
    leram = int(detalhes.findAll('div',{'class':'bar'})[5].findAll('a')[1].get_text().replace('.',''))
    
    
    try:
        generos = detalhes.find(id='livro-perfil-sinopse-txt').find('span').get_text()
    except:
        generos = None
    if generos != None:
        descricao = detalhes.find(id='livro-perfil-sinopse-txt').find('p').get_text().split(generos)[0]
        generos = generos.strip().split(' / ')
    else:
        try:
            descricao = detalhes.find(id='livro-perfil-sinopse-txt').find('p').get_text()
        except:
            descricao = None
    
    
    return(descricao, generos, Rating, Avaliacoes, total_resenhas_real, abandonos, relendo, querem_ler, lendo, leram, url_livro)

In [4]:
def data_de_lancamento(i, lancamento):
    url_edicoes = 'https://www.skoob.com.br/livro/edicoes/'+str(i)
    
    response3 = requests.get(url_edicoes)
    pagina_html3 = BeautifulSoup(response3.text)
    corpo3 = pagina_html3.findAll(id="corpo")[0].findAll('div',{'style':'margin-top:10px;'})[0] # corpo da página a ser feita a raspagem de dados (Web Scraping)
    
    numero_edicoes = len(corpo3.findAll('div',{'style':'float:left; width:180px;'}))
    lancamentos = []
    for edicoes in range(0,numero_edicoes):
        lancamentos.append(int(corpo3.findAll('div',{'style':'float:left; width:180px;'})[edicoes].get_text().split('Ano: ')[-1].split('Páginas:')[0]))
    try:
        if min(lancamentos) < lancamento:
            lancamento = min(lancamentos)
    except: 
        pass
    return(lancamento)

In [5]:
def obter_resenhas(i, Livro, total_resenhas, limite_resenhas, imprimir):
    
    lista_reviews = [] # lista com output de todas as reviews coletadas
    
    url_resenha = 'https://www.skoob.com.br/livro/resenhas/{}/mais-gostaram/mpage:{}'
    
    total_pags = math.ceil(total_resenhas/15)
    
    if limite_resenhas == 0: # se limite for definido como 0, pegará todas os comentários existentes
        pass   
    elif total_pags > math.ceil(limite_resenhas/15): # se houverem mais páginas que as necessárias para encontrar o limite
        total_pags = math.ceil(limite_resenhas/15) # limitará as páginas de modo a limitar o total de comentários
    else:
        pass # se limite_resenhas não for 0 nem maior que o existente, teremos menos reviews que o desejado, portanto pegaremos todos os disponíveis
    
    contador_reviews  = 0 # definindo um contador de comentários por livro
        
    for paginas_de_resenha in range(1,total_pags+1): # range das páginas com comentários, seguindo limite caso tenha sido definido
        response4 = requests.get(url_resenha.format(i,paginas_de_resenha))
        pagina_html4 = BeautifulSoup(response4.text)
        corpo4 = pagina_html4.findAll(id="corpo")[0] # corpo das reviews de cada página  
        
        
        if imprimir: # se True, irá printar o livro e página que está sendo coletado
            print('Livro {}: {} - Página: {}'.format(i, Livro, paginas_de_resenha)) # printando página da review, para identificar casos de bug

        # existem até 15 reviews por página, cada uma com o ID da review aparecendo 2x, portanto
        # o range padrão de coleta de reviews por página vai de 0 a 29, pulando de 2 em 2
        # no entanto, possívelmente ele foi limitado    
    
        if (paginas_de_resenha == math.ceil(limite_resenhas/15) and limite_resenhas!= 0): # se a página atual for a mesma definida ao limitar o numero de reviews
            ultima_review = (limite_resenhas - contador_reviews)*2 # encontrar quantas reviews ainda serão coletadas para atingir limite definido
            # multiplicar por 2 uma vez que pularemos de 2 em 2    
        else:
            ultima_review = 30 # se nao estiver no limite definido, usará o range completo das reviews 
            
        for resenhas in range(0,ultima_review,2):
            try: # testar se existe reviews nesta posição, uma vez que se o total não tiver sido limitado,
                # na última página de reviews pode haver menos de 15 reviews
                if imprimir: # se True, irá printar o livro e página e review que está sendo coletado
                    print('Livro {}: {} - Página: {} - Review: {}'.format(i, Livro, paginas_de_resenha,int(resenhas/2+1))) # printando a posição da review, para identificar casos de bug
                          
                # dados gerais da resenha    
                review_pt1 = corpo4.findAll(id="perfil-conteudo-intern")[0].findAll("div", id=lambda value: value and value.startswith("resenha"))[resenhas]
                id_resenha = review_pt1.get('id') # ID da review
                id_usuario = review_pt1.find('a').get('href').split('/')[-1] # ID do usuário
                nota = float(review_pt1.find('star-rating').get('rate')) # Nota dada pelo usuário
                           
                # dados de texto da resenha
                review_pt2 = corpo4.findAll(id="perfil-conteudo-intern")[0].findAll("div", id=lambda value: value and value.startswith("resenha"))[resenhas+1]
                            
                try: # caso exista um título, salvar ele separadamente
                    titulo = review_pt2.findAll('strong')[1].get_text() 
                except IndexError: # caso não exista um título, salvar None
                    titulo = None    
                    
                resenha = review_pt2.get_text() # obtendo o texto que consta título, data e resenha
                # na estrutura obtida, caso exista um titulo, ele é a ultima informação antes da resenha em si
                # caso não exista um título, a data é a ultima informação                
    
                if titulo == None: # caso não haja um título, usar a data como divisor para obter apenas a resenha
                    resenha = resenha.split('/',2)[-1][4:]
                else: # caso haja um título, utilizá-lo como divisor para obter a resenha
                    resenha = resenha.split(titulo,1)[-1]    
                livro_pagina_posicao = '{}-{}-{}'.format(i,paginas_de_resenha,(resenhas/2+1))
                
                lista_reviews.append({'review_id': id_resenha,
                                      'user_id': id_usuario,
                                      'livro_pagina_posicao':livro_pagina_posicao,
                                      'rating': nota,
                                      'review_title': titulo,
                                      'review': resenha})
            except:
                break
                
    return(lista_reviews)

In [6]:
def raspar_skoob(livro_inicial, livro_final, limite_resenhas,imprimir):
    # livro_inicial: primeiro livro a ser coletado (número inteiro não nulo)
    # livro_final: ultimo livro a ser coletado, incluindo-o (sendo necessáriamente um número maior que o do livro_inicial)
    # limite_resenhas: limite de reviews a serem coletados. 
        # Caso o valor seja menor que o número de reviews existentes, ele limitárá o total
        # Caso insira um valor maior que o número de reviews disponíveis, ele pegará o máximo existente
        # Caso não deseje limitar, inserir o valor 0
    # imprimir: se True, irá printar os estágios, se False, não irá printar os estágios
    
    
    contagem = 0 # contagem para definir saída do loop caso não existam 15 livros em sequência, um indício de que chegou ao final dos livros existentes
    
    lista_livros = [] # lista dos livros coletados    
    
    
    # range da lista de livros, começando no livro_inicial até livro_final, em que será feita a coleta
    for i in range(livro_inicial,livro_final+1):     
    
        if contagem == 15:
            print('\nHá {} tentativas não foi encontrado um livro, portanto, scraping será interrompido!'.format(contagem))
            break

        while True: # enquanto for true, seguira tentando obter os dados daquele único livro i
            try: # testar se existe um livro naquela posição, caso não exista simplesmente passará para o próximo livro
                lista_reviews=[]
                contagem = 0 # se o try funcionar, foi encontrado um livro, portanto, zerar contador
                
                # chamar a função dados_iniciais_livro
                Livro, Autor, ISBN_13, lancamento, Paginas, total_resenhas, url_resenha = dados_iniciais_livro(i)
                
                # chamar a função detalhes_do_livro
                descricao, generos, Rating, Avaliacoes, total_resenhas_real, abandonos, relendo, querem_ler, lendo, leram, url_livro = detalhes_do_livro(i, Livro) 
                
                # chamar a função da data_de_lancamento
                lancamento = data_de_lancamento(i, lancamento)                                             
                
                if total_resenhas != 0: # caso existam resenhas para este livro
                    lista_reviews = obter_resenhas(i, Livro, total_resenhas, limite_resenhas, imprimir)                      
                    
                else: # caso não exista nenhuma resenha 
                    lista_reviews = None
                # Salvar o livro após coleta
                lista_livros.append({'autor':Autor,
                                     'titulo':Livro,
                                     'numero_do_livro':i,
                                     'generos':generos,
                                     'data_lancamento':lancamento,
                                     'ISBN_13':ISBN_13,
                                     'paginas':Paginas,
                                     'nota_media':Rating,
                                     'total_de_avaliacoes':Avaliacoes,
                                     'leram':leram,
                                     'lendo':lendo,
                                     'querem_ler':querem_ler,
                                     'relendo':relendo,
                                     'abandonos':abandonos,
                                     'total_resenhas':total_resenhas_real,
                                     'url_livro':url_livro,
                                     'url_resenha':url_resenha,
                                     'descricao':descricao,
                                     'reviews':lista_reviews
                                    })
                break
                    
                    
            except IndexError: # caso não exista um livro nesta página, passar para próximo livro
                contagem +=1 # adicionar 1 ao contador para cada livro não existente
                break
                
            except requests.exceptions.ConnectionError: # caso dê erro de internet, continuar tentando até a internet voltar
                continue        
            except requests.exceptions.ChunkedEncodingError: # caso dê erro de internet, continuar tentando até a internet voltar
                continue           
       #     except:
       #         print('Um erro ocorreu no livro {}'.format(i))
       #         contagem +=1
       #         break
    print('\n')
    print (time.asctime(time.localtime(time.time())))    
    return(lista_livros)

In [7]:
print (time.asctime(time.localtime(time.time())))

Sun Mar 13 20:21:19 2022


In [ ]:
livros = raspar_skoob(1,10000,0,True)

Livro 1: Ensaio Sobre a Cegueira
Livro 1: Ensaio Sobre a Cegueira - Página: 1
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 1
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 2
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 3
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 4
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 5
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 6
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 7
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 8
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 9
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 10
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 11
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 12
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 13
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 14
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 15
Livro 1: Ensaio Sobre a Cegueira - Página: 2
Livro 1: Ensaio 

Livro 1: Ensaio Sobre a Cegueira - Página: 11
Livro 1: Ensaio Sobre a Cegueira - Página: 11 - Review: 1
Livro 1: Ensaio Sobre a Cegueira - Página: 11 - Review: 2
Livro 1: Ensaio Sobre a Cegueira - Página: 11 - Review: 3
Livro 1: Ensaio Sobre a Cegueira - Página: 11 - Review: 4
Livro 1: Ensaio Sobre a Cegueira - Página: 11 - Review: 5
Livro 1: Ensaio Sobre a Cegueira - Página: 11 - Review: 6
Livro 1: Ensaio Sobre a Cegueira - Página: 11 - Review: 7
Livro 1: Ensaio Sobre a Cegueira - Página: 11 - Review: 8
Livro 1: Ensaio Sobre a Cegueira - Página: 11 - Review: 9
Livro 1: Ensaio Sobre a Cegueira - Página: 11 - Review: 10
Livro 1: Ensaio Sobre a Cegueira - Página: 11 - Review: 11
Livro 1: Ensaio Sobre a Cegueira - Página: 11 - Review: 12
Livro 1: Ensaio Sobre a Cegueira - Página: 11 - Review: 13
Livro 1: Ensaio Sobre a Cegueira - Página: 11 - Review: 14
Livro 1: Ensaio Sobre a Cegueira - Página: 11 - Review: 15
Livro 1: Ensaio Sobre a Cegueira - Página: 12
Livro 1: Ensaio Sobre a Cegueira

Livro 1: Ensaio Sobre a Cegueira - Página: 20
Livro 1: Ensaio Sobre a Cegueira - Página: 20 - Review: 1
Livro 1: Ensaio Sobre a Cegueira - Página: 20 - Review: 2
Livro 1: Ensaio Sobre a Cegueira - Página: 20 - Review: 3
Livro 1: Ensaio Sobre a Cegueira - Página: 20 - Review: 4
Livro 1: Ensaio Sobre a Cegueira - Página: 20 - Review: 5
Livro 1: Ensaio Sobre a Cegueira - Página: 20 - Review: 6
Livro 1: Ensaio Sobre a Cegueira - Página: 20 - Review: 7
Livro 1: Ensaio Sobre a Cegueira - Página: 20 - Review: 8
Livro 1: Ensaio Sobre a Cegueira - Página: 20 - Review: 9
Livro 1: Ensaio Sobre a Cegueira - Página: 20 - Review: 10
Livro 1: Ensaio Sobre a Cegueira - Página: 20 - Review: 11
Livro 1: Ensaio Sobre a Cegueira - Página: 20 - Review: 12
Livro 1: Ensaio Sobre a Cegueira - Página: 20 - Review: 13
Livro 1: Ensaio Sobre a Cegueira - Página: 20 - Review: 14
Livro 1: Ensaio Sobre a Cegueira - Página: 20 - Review: 15
Livro 1: Ensaio Sobre a Cegueira - Página: 21
Livro 1: Ensaio Sobre a Cegueira

Livro 1: Ensaio Sobre a Cegueira - Página: 29
Livro 1: Ensaio Sobre a Cegueira - Página: 29 - Review: 1
Livro 1: Ensaio Sobre a Cegueira - Página: 29 - Review: 2
Livro 1: Ensaio Sobre a Cegueira - Página: 29 - Review: 3
Livro 1: Ensaio Sobre a Cegueira - Página: 29 - Review: 4
Livro 1: Ensaio Sobre a Cegueira - Página: 29 - Review: 5
Livro 1: Ensaio Sobre a Cegueira - Página: 29 - Review: 6
Livro 1: Ensaio Sobre a Cegueira - Página: 29 - Review: 7
Livro 1: Ensaio Sobre a Cegueira - Página: 29 - Review: 8
Livro 1: Ensaio Sobre a Cegueira - Página: 29 - Review: 9
Livro 1: Ensaio Sobre a Cegueira - Página: 29 - Review: 10
Livro 1: Ensaio Sobre a Cegueira - Página: 29 - Review: 11
Livro 1: Ensaio Sobre a Cegueira - Página: 29 - Review: 12
Livro 1: Ensaio Sobre a Cegueira - Página: 29 - Review: 13
Livro 1: Ensaio Sobre a Cegueira - Página: 29 - Review: 14
Livro 1: Ensaio Sobre a Cegueira - Página: 29 - Review: 15
Livro 1: Ensaio Sobre a Cegueira - Página: 30
Livro 1: Ensaio Sobre a Cegueira

Livro 1: Ensaio Sobre a Cegueira - Página: 38
Livro 1: Ensaio Sobre a Cegueira - Página: 38 - Review: 1
Livro 1: Ensaio Sobre a Cegueira - Página: 38 - Review: 2
Livro 1: Ensaio Sobre a Cegueira - Página: 38 - Review: 3
Livro 1: Ensaio Sobre a Cegueira - Página: 38 - Review: 4
Livro 1: Ensaio Sobre a Cegueira - Página: 38 - Review: 5
Livro 1: Ensaio Sobre a Cegueira - Página: 38 - Review: 6
Livro 1: Ensaio Sobre a Cegueira - Página: 38 - Review: 7
Livro 1: Ensaio Sobre a Cegueira - Página: 38 - Review: 8
Livro 1: Ensaio Sobre a Cegueira - Página: 38 - Review: 9
Livro 1: Ensaio Sobre a Cegueira - Página: 38 - Review: 10
Livro 1: Ensaio Sobre a Cegueira - Página: 38 - Review: 11
Livro 1: Ensaio Sobre a Cegueira - Página: 38 - Review: 12
Livro 1: Ensaio Sobre a Cegueira - Página: 38 - Review: 13
Livro 1: Ensaio Sobre a Cegueira - Página: 38 - Review: 14
Livro 1: Ensaio Sobre a Cegueira - Página: 38 - Review: 15
Livro 1: Ensaio Sobre a Cegueira - Página: 39
Livro 1: Ensaio Sobre a Cegueira

...

Livro 47: Os Homens Que Não Amavam as Mulheres - Página: 70 - Review: 15
Livro 47: Os Homens Que Não Amavam as Mulheres - Página: 71
Livro 47: Os Homens Que Não Amavam as Mulheres - Página: 71 - Review: 1
Livro 47: Os Homens Que Não Amavam as Mulheres - Página: 71 - Review: 2
Livro 47: Os Homens Que Não Amavam as Mulheres - Página: 71 - Review: 3
Livro 47: Os Homens Que Não Amavam as Mulheres - Página: 71 - Review: 4
Livro 47: Os Homens Que Não Amavam as Mulheres - Página: 71 - Review: 5
Livro 47: Os Homens Que Não Amavam as Mulheres - Página: 71 - Review: 6
Livro 47: Os Homens Que Não Amavam as Mulheres - Página: 71 - Review: 7
Livro 47: Os Homens Que Não Amavam as Mulheres - Página: 71 - Review: 8
Livro 47: Os Homens Que Não Amavam as Mulheres - Página: 71 - Review: 9
Livro 47: Os Homens Que Não Amavam as Mulheres - Página: 71 - Review: 10
Livro 47: Os Homens Que Não Amavam as Mulheres - Página: 71 - Review: 11
Livro 47: Os Homens Que Não Amavam as Mulheres - Página: 71 - Review: 12


Livro 48: Lua Nova - Página: 10 - Review: 14
Livro 48: Lua Nova - Página: 10 - Review: 15
Livro 48: Lua Nova - Página: 11
Livro 48: Lua Nova - Página: 11 - Review: 1
Livro 48: Lua Nova - Página: 11 - Review: 2
Livro 48: Lua Nova - Página: 11 - Review: 3
Livro 48: Lua Nova - Página: 11 - Review: 4
Livro 48: Lua Nova - Página: 11 - Review: 5
Livro 48: Lua Nova - Página: 11 - Review: 6
Livro 48: Lua Nova - Página: 11 - Review: 7
Livro 48: Lua Nova - Página: 11 - Review: 8
Livro 48: Lua Nova - Página: 11 - Review: 9
Livro 48: Lua Nova - Página: 11 - Review: 10
Livro 48: Lua Nova - Página: 11 - Review: 11
Livro 48: Lua Nova - Página: 11 - Review: 12
Livro 48: Lua Nova - Página: 11 - Review: 13
Livro 48: Lua Nova - Página: 11 - Review: 14
Livro 48: Lua Nova - Página: 11 - Review: 15
Livro 48: Lua Nova - Página: 12
Livro 48: Lua Nova - Página: 12 - Review: 1
Livro 48: Lua Nova - Página: 12 - Review: 2
Livro 48: Lua Nova - Página: 12 - Review: 3
Livro 48: Lua Nova - Página: 12 - Review: 4
Livr

Livro 48: Lua Nova - Página: 23
Livro 48: Lua Nova - Página: 23 - Review: 1
Livro 48: Lua Nova - Página: 23 - Review: 2
Livro 48: Lua Nova - Página: 23 - Review: 3
Livro 48: Lua Nova - Página: 23 - Review: 4
Livro 48: Lua Nova - Página: 23 - Review: 5
Livro 48: Lua Nova - Página: 23 - Review: 6
Livro 48: Lua Nova - Página: 23 - Review: 7
Livro 48: Lua Nova - Página: 23 - Review: 8
Livro 48: Lua Nova - Página: 23 - Review: 9
Livro 48: Lua Nova - Página: 23 - Review: 10
Livro 48: Lua Nova - Página: 23 - Review: 11
Livro 48: Lua Nova - Página: 23 - Review: 12
Livro 48: Lua Nova - Página: 23 - Review: 13
Livro 48: Lua Nova - Página: 23 - Review: 14
Livro 48: Lua Nova - Página: 23 - Review: 15
Livro 48: Lua Nova - Página: 24
Livro 48: Lua Nova - Página: 24 - Review: 1
Livro 48: Lua Nova - Página: 24 - Review: 2
Livro 48: Lua Nova - Página: 24 - Review: 3
Livro 48: Lua Nova - Página: 24 - Review: 4
Livro 48: Lua Nova - Página: 24 - Review: 5
Livro 48: Lua Nova - Página: 24 - Review: 6
Livro 

Livro 48: Lua Nova - Página: 35
Livro 48: Lua Nova - Página: 35 - Review: 1
Livro 48: Lua Nova - Página: 35 - Review: 2
Livro 48: Lua Nova - Página: 35 - Review: 3
Livro 48: Lua Nova - Página: 35 - Review: 4
Livro 48: Lua Nova - Página: 35 - Review: 5
Livro 48: Lua Nova - Página: 35 - Review: 6
Livro 48: Lua Nova - Página: 35 - Review: 7
Livro 48: Lua Nova - Página: 35 - Review: 8
Livro 48: Lua Nova - Página: 35 - Review: 9
Livro 48: Lua Nova - Página: 35 - Review: 10
Livro 48: Lua Nova - Página: 35 - Review: 11
Livro 48: Lua Nova - Página: 35 - Review: 12
Livro 48: Lua Nova - Página: 35 - Review: 13
Livro 48: Lua Nova - Página: 35 - Review: 14
Livro 48: Lua Nova - Página: 35 - Review: 15
Livro 48: Lua Nova - Página: 36
Livro 48: Lua Nova - Página: 36 - Review: 1
Livro 48: Lua Nova - Página: 36 - Review: 2
Livro 48: Lua Nova - Página: 36 - Review: 3
Livro 48: Lua Nova - Página: 36 - Review: 4
Livro 48: Lua Nova - Página: 36 - Review: 5
Livro 48: Lua Nova - Página: 36 - Review: 6
Livro 

Livro 48: Lua Nova - Página: 46 - Review: 14
Livro 48: Lua Nova - Página: 46 - Review: 15
Livro 48: Lua Nova - Página: 47
Livro 48: Lua Nova - Página: 47 - Review: 1
Livro 48: Lua Nova - Página: 47 - Review: 2
Livro 48: Lua Nova - Página: 47 - Review: 3
Livro 48: Lua Nova - Página: 47 - Review: 4
Livro 48: Lua Nova - Página: 47 - Review: 5
Livro 48: Lua Nova - Página: 47 - Review: 6
Livro 48: Lua Nova - Página: 47 - Review: 7
Livro 48: Lua Nova - Página: 47 - Review: 8
Livro 48: Lua Nova - Página: 47 - Review: 9
Livro 48: Lua Nova - Página: 47 - Review: 10
Livro 48: Lua Nova - Página: 47 - Review: 11
Livro 48: Lua Nova - Página: 47 - Review: 12
Livro 48: Lua Nova - Página: 47 - Review: 13
Livro 48: Lua Nova - Página: 47 - Review: 14
Livro 48: Lua Nova - Página: 47 - Review: 15


In [ ]:
df_livros = pd.DataFrame(livros)

In [ ]:
df_livros.to_csv('livros_1_10000')